<h1 style="text-align: center; font-size: 60px;">Assignment 12</h1>
<h3 style="text-align: center; font-size: 20px;">(submitted on 17th Sept 2025)</h3>

# ENSEMBLE LEARNING: BAGGING

In [1]:
import pandas as pd

df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [3]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
df.Outcome.value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

### Train test split

In [5]:
X = df.drop("Outcome",axis="columns")
y = df.Outcome

In [6]:
# Standard Scaling the data to normalize

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:3]

array([[ 0.63994726,  0.84832379,  0.14964075,  0.90726993, -0.69289057,
         0.20401277,  0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575,  0.53090156, -0.69289057,
        -0.68442195, -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, -1.28821221, -0.69289057,
        -1.10325546,  0.60439732, -0.10558415]])

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=10)

In [8]:
X_train.shape

(576, 8)

In [9]:
X_test.shape

(192, 8)

In [10]:
y_train.value_counts()

Outcome
0    375
1    201
Name: count, dtype: int64

In [11]:
# ratio
201/375

0.536

In [12]:
y_test.value_counts()

Outcome
0    125
1     67
Name: count, dtype: int64

In [13]:
67/125

0.536

### Train using stand alone model

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)
scores

array([0.66883117, 0.67532468, 0.64935065, 0.79738562, 0.73856209])

In [15]:
scores.mean()

np.float64(0.7058908411849588)

### Train using Bagging

In [17]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
bag_model.fit(X_train, y_train)
bag_model.oob_score_

0.7534722222222222

In [18]:
bag_model.score(X_test, y_test)

0.7760416666666666

In [19]:
bag_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
scores = cross_val_score(bag_model, X, y, cv=5)
scores

array([0.75324675, 0.72727273, 0.74675325, 0.82352941, 0.73856209])

In [20]:
scores.mean()

np.float64(0.7578728461081402)

### Train using Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(n_estimators=50), X, y, cv=5)
scores.mean()

np.float64(0.7735166793990323)

## EXERCISE

In [22]:
'''
1. Load heart disease dataset in pandas dataframe
2. Remove outliers using Z score. Usual guideline is to remove anything that has Z score > 3 formula or Z score < -3
3. Convert text columns to numbers using label encoding and one hot encoding
4. Apply scaling
5. Build a classification model using support vector machine. Use standalone model as well as Bagging model and 
   check if you see any difference in the performance.
6. Now use decision tree classifier. Use standalone model as well as Bagging and check if you notice any difference in performance
7. Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why. 
   Use internet to figure out in what conditions bagging works the best.
'''

'\n1. Load heart disease dataset in pandas dataframe\n2. Remove outliers using Z score. Usual guideline is to remove anything that has Z score > 3 formula or Z score < -3\n3. Convert text columns to numbers using label encoding and one hot encoding\n4. Apply scaling\n5. Build a classification model using support vector machine. Use standalone model as well as Bagging model and \n   check if you see any difference in the performance.\n6. Now use decision tree classifier. Use standalone model as well as Bagging and check if you notice any difference in performance\n7. Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why. \n   Use internet to figure out in what conditions bagging works the best.\n'

In [32]:
df = pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [33]:
df.shape

(918, 12)

In [83]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [84]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [85]:
# So the Outlier has an abnormal value compared to others.

### Treat Outliers

In [78]:
# using Z score > 3
df[df.Cholesterol > (df.Cholesterol.mean() + 3 * df.Cholesterol.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
76,32,M,ASY,118,529,0,Normal,130,N,0.0,Flat,1
149,54,M,ASY,130,603,1,Normal,125,Y,1.0,Flat,1
616,67,F,NAP,115,564,0,LVH,160,N,1.6,Flat,0


In [37]:
df.shape

(918, 12)

In [79]:
df1 = df[df.Cholesterol <= (df.Cholesterol.mean() + 3 * df.Cholesterol.std())]
df1.shape

(915, 12)

In [80]:
df[df.MaxHR > (df.MaxHR.mean() + 3 * df.MaxHR.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease


In [42]:
df[df.FastingBS > (df.FastingBS.mean() + 3 * df.FastingBS.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease


In [45]:
df[df.Oldpeak > (df.Oldpeak.mean() + 3 * df.Oldpeak.std())]
# is an outlier

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
166,50,M,ASY,140,231,0,ST,140,Y,5.0,Flat,1
702,59,M,TA,178,270,0,LVH,145,N,4.2,Down,0
771,55,M,ASY,140,217,0,Normal,111,Y,5.6,Down,1
791,51,M,ASY,140,298,0,Normal,122,Y,4.2,Flat,1
850,62,F,ASY,160,164,0,LVH,145,N,6.2,Down,1
900,58,M,ASY,114,318,0,ST,140,N,4.4,Down,1


In [46]:
# Then change
df2 = df1[df1.Oldpeak <= (df1.Oldpeak.mean() + 3 * df1.Oldpeak.std())]
df2.shape

(909, 12)

In [81]:
df[df.RestingBP > (df.RestingBP.mean() + 3 * df.RestingBP.std())]
# is an outlier

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
109,39,M,ATA,190,241,0,Normal,106,N,0.0,Up,0
241,54,M,ASY,200,198,0,Normal,142,Y,2.0,Flat,1
365,64,F,ASY,200,0,0,Normal,140,Y,1.0,Flat,1
399,61,M,NAP,200,0,1,ST,70,N,0.0,Flat,1
592,61,M,ASY,190,287,1,LVH,150,Y,2.0,Down,1
732,56,F,ASY,200,288,1,LVH,133,Y,4.0,Down,1
759,54,M,ATA,192,283,0,LVH,195,N,0.0,Up,1


In [82]:
# Then change
df3 = df2[df2.RestingBP <= (df2.RestingBP.mean() + 3 * df2.RestingBP.std())]
df3.shape

(908, 12)

In [54]:
df.ChestPainType.unique()

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [55]:
df.RestingECG.unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [56]:
df.ExerciseAngina.unique()

array(['N', 'Y'], dtype=object)

In [57]:
df.ST_Slope.unique()

array(['Up', 'Flat', 'Down'], dtype=object)

### Handle text columns using label encoding and one hot encoding

In [59]:
df4 = df3.copy()

df4['ExerciseAngina'] = df4['ExerciseAngina'].replace({'N': 0, 'Y': 1})
df4['ST_Slope'] = df4['ST_Slope'].replace({'Down': 1, 'Flat': 2, 'Up': 3})
df4['RestingECG'] = df4['RestingECG'].replace({'Normal': 1, 'ST': 2, 'LVH': 3})

df4.head()

C:\Users\Pranav S P\AppData\Local\Temp\ipykernel_21768\1013741484.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df4['ExerciseAngina'] = df4['ExerciseAngina'].replace({'N': 0, 'Y': 1})
C:\Users\Pranav S P\AppData\Local\Temp\ipykernel_21768\1013741484.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df4['ST_Slope'] = df4['ST_Slope'].replace({'Down': 1, 'Flat': 2, 'Up': 3})
C:\Users\Pranav S P\AppData\Local\Temp\ipykernel_21768\1013741484.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be remove

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,1,172,0,0.0,3,0
1,49,F,NAP,160,180,0,1,156,0,1.0,2,1
2,37,M,ATA,130,283,0,2,98,0,0.0,3,0
3,48,F,ASY,138,214,0,1,108,1,1.5,2,1
4,54,M,NAP,150,195,0,1,122,0,0.0,3,0


In [60]:
df5 = pd.get_dummies(df4, drop_first=True)
df5.head()

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,140,289,0,1,172,0,0.0,3,0,True,True,False,False
1,49,160,180,0,1,156,0,1.0,2,1,False,False,True,False
2,37,130,283,0,2,98,0,0.0,3,0,True,True,False,False
3,48,138,214,0,1,108,1,1.5,2,1,False,False,False,False
4,54,150,195,0,1,122,0,0.0,3,0,True,False,True,False


In [61]:
X = df5.drop("HeartDisease",axis='columns')
y = df5.HeartDisease

X.head()

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,140,289,0,1,172,0,0.0,3,True,True,False,False
1,49,160,180,0,1,156,0,1.0,2,False,False,True,False
2,37,130,283,0,2,98,0,0.0,3,True,True,False,False
3,48,138,214,0,1,108,1,1.5,2,False,False,False,False
4,54,150,195,0,1,122,0,0.0,3,True,False,True,False


In [62]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-1.43407402,  0.45460383,  0.85066047, ...,  2.07603986,
        -0.53319777, -0.23100708],
       [-0.4782586 ,  1.58318969, -0.16483361, ..., -0.48168632,
         1.87547672, -0.23100708],
       [-1.75267916, -0.1096891 ,  0.79476171, ...,  2.07603986,
        -0.53319777, -0.23100708],
       ...,
       [ 0.37135511, -0.1096891 , -0.62134013, ..., -0.48168632,
        -0.53319777, -0.23100708],
       [ 0.37135511, -0.1096891 ,  0.35688812, ...,  2.07603986,
        -0.53319777, -0.23100708],
       [-1.64647745,  0.34174525, -0.21141591, ..., -0.48168632,
         1.87547672, -0.23100708]], shape=(908, 13))

In [63]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=20)

In [64]:
X_train.shape

(726, 13)

In [65]:
X_test.shape

(182, 13)

### Train a model using standalone support vector machine.

In [71]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

scores = cross_val_score(SVC(), X_scaled, y, cv=5)
scores.mean()

np.float64(0.8149231983486127)

#### Use bagging now with svm

In [72]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(estimator=SVC(), n_estimators=100, max_samples=0.8, random_state=0)
scores = cross_val_score(bag_model, X_scaled, y, cv=5)
scores.mean()

np.float64(0.8160220994475138)

In [74]:
# CONCLUSION:
# using bagging in case of SVM doesn't make much difference in terms of model accuracy.

### Train a model using decision tree and then using bagging

In [75]:
from sklearn.tree import DecisionTreeClassifier

scores = cross_val_score(DecisionTreeClassifier(random_state=0), X_scaled, y, cv=5)
scores.mean()

np.float64(0.7311699350373384)

#### Use bagging now with decision tree

In [76]:
bag_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(random_state=0), 
    n_estimators=100, 
    max_samples=0.9, 
    oob_score=True,
    random_state=0
)

scores = cross_val_score(bag_model, X_scaled, y, cv=5)
scores.mean()

np.float64(0.7972800679983)

In [87]:
# Bagging score improved from 73.11% to 79.72% for decision tree classifier.

# END OF ASSIGNMENT 12